In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [2]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,turkcell'den elimi ayağımı çektiğim iyi oldu,0
1,"avrasya tüneli inşaatı sırasında, yerin 106 me...",1
2,salla kazan yeniden aramızda ?????? hazırkart ...,1
3,turkcell mukemmele yakin kalitede musteri tems...,1
4,o deilde bugün turkcell hiç msj atmadı amk kes...,0
...,...,...
9121,oha bu turkcell reklamındaki bıdıkları yerim y...,1
9122,ulan turkcell lazım olduğunda dakika gönderme ...,0
9123,en beğenilen şirket ödülü yine turkcell'in,1
9124,can yücel: bağlanmayacaksın. turkcell superonl...,0


In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [6]:
def tf_idf (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    tokenizer = Tokenizer(num_words=1000) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    vocabulary=[]
    for key in word_index.keys():
        vocabulary.append(key)

    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                     ('tfid', TfidfTransformer())]).fit(X_train)

    res = dict(zip(vocabulary, pipe['tfid'].idf_))

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    
    skipped_words = 0
    embedding_dim = 1 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = res[word] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector 
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [7]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= tf_idf (X_train,y_train,X_test,y_test)
    
    modelCNNLSTM = Sequential() 
    pat = 5 #this is the number of epochs with no improvment after which the training will stop
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)
    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=256, kernel_size=5, activation='relu'))
    modelCNNLSTM.add(MaxPooling1D(pool_size=4))
    modelCNNLSTM.add(Dropout(0.25))
    modelCNNLSTM.add(LSTM(128))                            
    modelCNNLSTM.add(Flatten())
    modelCNNLSTM.add(Dense(1, activation='sigmoid'))
    modelCNNLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNNLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6956 - acc: 0.5107 - precision: 0.5079 - recall: 0.5428
Epoch 2/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6919 - acc: 0.5243 - precision: 0.5249 - recall: 0.4667
Epoch 3/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6914 - acc: 0.5253 - precision: 0.5234 - recall: 0.5164
Epoch 4/50
257/257 [==============================] - 2s 10ms/step - loss: 0.6903 - acc: 0.5267 - precision: 0.5234 - recall: 0.5501
Epoch 5/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6897 - acc: 0.5304 - precision: 0.5287 - recall: 0.5205
Epoch 6/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6898 - acc: 0.5337 - precision: 0.5312 - recall: 0.5377 1s - loss: 0.6880 
Epoch 7/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6887 - ac

257/257 [==============================] - 2s 8ms/step - loss: 0.6912 - acc: 0.5292 - precision_3: 0.5254 - recall_3: 0.6007
Epoch 8/50
257/257 [==============================] - 2s 8ms/step - loss: 0.6902 - acc: 0.5320 - precision_3: 0.5292 - recall_3: 0.5759
Epoch 9/50
257/257 [==============================] - 2s 8ms/step - loss: 0.6903 - acc: 0.5299 - precision_3: 0.5251 - recall_3: 0.6214
Epoch 10/50
257/257 [==============================] - 2s 8ms/step - loss: 0.6897 - acc: 0.5305 - precision_3: 0.5273 - recall_3: 0.5859
Epoch 11/50
257/257 [==============================] - 2s 9ms/step - loss: 0.6893 - acc: 0.5312 - precision_3: 0.5270 - recall_3: 0.6071
Epoch 12/50
257/257 [==============================] - 2s 9ms/step - loss: 0.6887 - acc: 0.5343 - precision_3: 0.5312 - recall_3: 0.5808
Epoch 13/50
257/257 [==============================] - 2s 8ms/step - loss: 0.6884 - acc: 0.5345 - precision_3: 0.5293 - recall_3: 0.6205
Epoch 14/50
257/257 [==============================] - 

257/257 [==============================] - 2s 10ms/step - loss: 0.6764 - acc: 0.5696 - precision_5: 0.5700 - recall_5: 0.5787
Epoch 23/50
257/257 [==============================] - 2s 9ms/step - loss: 0.6760 - acc: 0.5659 - precision_5: 0.5638 - recall_5: 0.5960
Epoch 24/50
257/257 [==============================] - 2s 9ms/step - loss: 0.6753 - acc: 0.5645 - precision_5: 0.5631 - recall_5: 0.5894
Epoch 25/50
257/257 [==============================] - 3s 11ms/step - loss: 0.6739 - acc: 0.5693 - precision_5: 0.5689 - recall_5: 0.5853
Epoch 26/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6737 - acc: 0.5756 - precision_5: 0.5748 - recall_5: 0.5923
Epoch 27/50
257/257 [==============================] - 2s 10ms/step - loss: 0.6721 - acc: 0.5710 - precision_5: 0.5717 - recall_5: 0.5787
Epoch 28/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6698 - acc: 0.5766 - precision_5: 0.5783 - recall_5: 0.5773
Epoch 29/50
257/257 [===========================

257/257 [==============================] - 3s 10ms/step - loss: 0.6528 - acc: 0.5951 - precision_7: 0.6013 - recall_7: 0.5695
Epoch 36/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6509 - acc: 0.6004 - precision_7: 0.6007 - recall_7: 0.6042
Epoch 37/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6446 - acc: 0.6056 - precision_7: 0.6104 - recall_7: 0.5884
Epoch 38/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6418 - acc: 0.6057 - precision_7: 0.6122 - recall_7: 0.5814 1s - loss: 0.6412 - acc:
Epoch 39/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6363 - acc: 0.6178 - precision_7: 0.6161 - recall_7: 0.6300
Epoch 40/50
257/257 [==============================] - 3s 11ms/step - loss: 0.6369 - acc: 0.6113 - precision_7: 0.6165 - recall_7: 0.5933
Epoch 41/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6311 - acc: 0.6170 - precision_7: 0.6173 - recall_7: 0.6203
Epoch 42/50
257/257 [

257/257 [==============================] - 2s 10ms/step - loss: 0.6047 - acc: 0.6401 - precision_9: 0.6400 - recall_9: 0.6397
Epoch 49/50
257/257 [==============================] - 3s 10ms/step - loss: 0.6011 - acc: 0.6477 - precision_9: 0.6462 - recall_9: 0.6521
Epoch 50/50
29/29 [==============================] - 0s 5ms/step - loss: 0.7500 - acc: 0.5274 - precision_9: 0.5380 - recall_9: 0.4170


In [8]:
def Average(lst):
    return sum(lst) / len(lst)

In [9]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.530687
test precision: 0.534600
test recall: 0.518228
test f1_score: 0.519433


In [ ]:
#word2vec

In [10]:
df

,tweets,duygu
0,yine mi cuma?????? (@ turkcell küçükyalı plaza...,1
1,lan olm ya ders seçerken net giderse neticede ...,1
2,turkcell sağolsun 4 gb internet hediye etti ar...,1
3,turkcell bin 799 köyü 4.5g ile tanıştıracak,1
4,"ah turkcell ah, beni bu şapşal uygulamalarınla...",0
...,...,...
9121,o değil artık turkcell de mesaj atmıyor.,0
9122,orospu çocuğu turkcell,0
9123,"internetsiz kaldim sinirsizdan , sinirliya gec...",0
9124,geçtiğimiz yıl serdar ortaç'ın bir şarkısını t...,0


In [11]:
X_t=df['tweets'].tolist()

In [12]:
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [13]:
from gensim.models import Word2Vec
import time
start=time.time()
n_features=32
window_size=5
min_count=1
epoch=50
n_workers=8

wordembeddings = Word2Vec(size = n_features,
            window = window_size, 
            min_count= min_count,
            workers = n_workers, 
            sg=1)
wordembeddings.build_vocab(X_t)
wordembeddings.train(X_t, 
            total_examples=wordembeddings.corpus_count,  
            epochs = epoch)
end=time.time()
print(end-start)

12.789406776428223


In [14]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [15]:
def word2vec (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 32 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak

        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [16]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= word2vec (X_train,y_train,X_test,y_test)
    
    modelLSTM = Sequential() 
    pat = 5
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)
    modelLSTM = Sequential()
    modelLSTM.add(embedding_layer)
    modelLSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    modelLSTM.add(Dense(1, activation='sigmoid'))
    modelLSTM.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelLSTM.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelLSTM.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

<ipython-input-15-7d0886ed6bc1>:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 19s 75ms/step - loss: 0.5223 - acc: 0.7441 - precision_10: 0.7507 - recall_10: 0.7313
Epoch 2/50
257/257 [==============================] - 20s 77ms/step - loss: 0.4817 - acc: 0.7672 - precision_10: 0.7744 - recall_10: 0.7544
Epoch 3/50
257/257 [==============================] - 21s 82ms/step - loss: 0.4601 - acc: 0.7795 - precision_10: 0.7875 - recall_10: 0.7659
Epoch 4/50
257/257 [==============================] - 19s 74ms/step - loss: 0.4431 - acc: 0.7905 - precision_10: 0.7997 - recall_10: 0.7754
Epoch 5/50
257/257 [==============================] - 19s 75ms/step - loss: 0.4319 - acc: 0.7953 - precision_10: 0.8058 - recall_10: 0.7785
Epoch 6/50
257/257 [==============================] - 20s 77ms/step - loss: 0.4210 - acc: 0.8038 - precision_10: 0.8102 - recall_10: 0.7939
Epoch 7/50
257/257 [==============================] - 20s 76ms/

257/257 [==============================] - 20s 76ms/step - loss: 0.3370 - acc: 0.8510 - precision_12: 0.8560 - recall_12: 0.8435
Epoch 13/50
257/257 [==============================] - 20s 80ms/step - loss: 0.3203 - acc: 0.8590 - precision_12: 0.8631 - recall_12: 0.8530
Epoch 14/50
257/257 [==============================] - 20s 79ms/step - loss: 0.3106 - acc: 0.8651 - precision_12: 0.8675 - recall_12: 0.8615
Epoch 15/50
257/257 [==============================] - 20s 78ms/step - loss: 0.2945 - acc: 0.8667 - precision_12: 0.8693 - recall_12: 0.8627
Epoch 16/50
257/257 [==============================] - 20s 79ms/step - loss: 0.2814 - acc: 0.8737 - precision_12: 0.8792 - recall_12: 0.8662
Epoch 17/50
257/257 [==============================] - 20s 77ms/step - loss: 0.2678 - acc: 0.8821 - precision_12: 0.8875 - recall_12: 0.8749
Epoch 18/50
257/257 [==============================] - 20s 76ms/step - loss: 0.2551 - acc: 0.8896 - precision_12: 0.8924 - recall_12: 0.8857
Epoch 19/50
257/257 [====

257/257 [==============================] - 19s 73ms/step - loss: 0.1967 - acc: 0.9173 - precision_14: 0.9190 - recall_14: 0.9161
Epoch 25/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1900 - acc: 0.9204 - precision_14: 0.9222 - recall_14: 0.9190
Epoch 26/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1742 - acc: 0.9299 - precision_14: 0.9338 - recall_14: 0.9261
Epoch 27/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1780 - acc: 0.9243 - precision_14: 0.9261 - recall_14: 0.9229
Epoch 28/50
257/257 [==============================] - 19s 75ms/step - loss: 0.1707 - acc: 0.9312 - precision_14: 0.9325 - recall_14: 0.9304
Epoch 29/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1617 - acc: 0.9346 - precision_14: 0.9367 - recall_14: 0.9328
Epoch 30/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1529 - acc: 0.9367 - precision_14: 0.9376 - recall_14: 0.9362
Epoch 31/50
257/257 [====

257/257 [==============================] - 18s 68ms/step - loss: 0.1234 - acc: 0.9520 - precision_16: 0.9538 - recall_16: 0.9501
Epoch 37/50
257/257 [==============================] - 18s 69ms/step - loss: 0.1292 - acc: 0.9485 - precision_16: 0.9513 - recall_16: 0.9455
Epoch 38/50
257/257 [==============================] - 18s 69ms/step - loss: 0.1295 - acc: 0.9480 - precision_16: 0.9528 - recall_16: 0.9428
Epoch 39/50
257/257 [==============================] - 18s 69ms/step - loss: 0.1248 - acc: 0.9498 - precision_16: 0.9490 - recall_16: 0.9508
Epoch 40/50
257/257 [==============================] - 18s 68ms/step - loss: 0.1250 - acc: 0.9487 - precision_16: 0.9504 - recall_16: 0.9469
Epoch 41/50
257/257 [==============================] - 18s 69ms/step - loss: 0.1188 - acc: 0.9541 - precision_16: 0.9549 - recall_16: 0.9533
Epoch 42/50
257/257 [==============================] - 18s 71ms/step - loss: 0.1141 - acc: 0.9500 - precision_16: 0.9518 - recall_16: 0.9479
Epoch 43/50
257/257 [====

257/257 [==============================] - 18s 69ms/step - loss: 0.0987 - acc: 0.9623 - precision_18: 0.9615 - recall_18: 0.9629
Epoch 49/50
257/257 [==============================] - 18s 69ms/step - loss: 0.0950 - acc: 0.9614 - precision_18: 0.9625 - recall_18: 0.9600
Epoch 50/50
29/29 [==============================] - 0s 11ms/step - loss: 0.8658 - acc: 0.7873 - precision_18: 0.8186 - recall_18: 0.7521
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/50
257/257 [==============================] - 17s 68ms/step - loss: 0.5212 - acc: 0.7440 - precision_19: 0.7531 - recall_19: 0.7216
Epoch 2/50
257/257 [==============================] - 18s 68ms/step - loss: 0.4763 - acc: 0.7714 - precision_19: 0.7814 - recall_19: 0.7500
Epoch 3/50
257/257 [==============================] - 18s 69ms/step - loss: 0.4617 - acc: 0.7796 - precision_19: 0.7909 - recall_19: 0.7569
Epoch 4/50
257/257 [==============================] - 18s 69ms/step - loss

In [17]:
def Average(lst):
    return sum(lst) / len(lst)

In [18]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.784572
test precision: 0.791544
test recall: 0.773285
test f1_score: 0.781916


In [ ]:
#fastText

In [6]:
df


,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1
4,"telefonumu modem olarak kullandığım doğrudur, ...",1
...,...,...
9135,artık soygunlarınızdan bıktım turkcell beni bi...,0
9136,turkcell kadar müşterisini mağdur eden bir fir...,0
9137,tam 20dk süren kabataş-kadıköy motor yolculuğu...,0
9138,"turkcell 7 yıldır kullandığım hattımda , kimli...",0


In [42]:
# Load my word embeddings
#import gensim
#wordembeddings = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/CNNPhraseEmbeddings/Resources/GoogleNews-vectors-negative300.bin', binary=True)

import fasttext
import fasttext.util
wordembeddings = fasttext.load_model('D:/Desktop/İndirilenler yedek/cc.tr.300.bin')

In [43]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [44]:
def fasttext (X_train,y_train,X_test,y_test):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    sequences_X_train = tokenizer.texts_to_sequences(X_train)
    word_index = tokenizer.word_index
    max_length = 0
    for review_number in range(len(sequences_X_train)):
        numberofwords=len(sequences_X_train[review_number])
        if (numberofwords) > (max_length):
            max_length = numberofwords

    X_train = pad_sequences(sequences_X_train, maxlen=max_length)
    y_train = np.asarray(y_train)

    sequences_X_test = tokenizer.texts_to_sequences(X_test)
    X_test = pad_sequences(sequences_X_test, maxlen=max_length)
    y_test = np.asarray(y_test)

    unique_words = len(word_index)
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 300
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items():
        try:
            embedding_vector = wordembeddings[word]
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
      
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [45]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
    
    pat = 5
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(128,activation='relu',return_sequences= True))
    model.add(SimpleRNN(256,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 28s 110ms/step - loss: 0.5593 - acc: 0.7092 - precision_31: 0.7119 - recall_31: 0.7068
Epoch 2/50
257/257 [==============================] - 28s 108ms/step - loss: 0.4488 - acc: 0.7975 - precision_31: 0.8072 - recall_31: 0.7839
Epoch 3/50
257/257 [==============================] - 28s 108ms/step - loss: 0.3872 - acc: 0.8337 - precision_31: 0.8435 - recall_31: 0.8210
Epoch 4/50
257/257 [==============================] - 28s 109ms/step - loss: 0.3454 - acc: 0.8547 - precision_31: 0.8677 - recall_31: 0.8385
Epoch 5/50
257/257 [==============================] - 28s 108ms/step - loss: 0.2966 - acc: 0.8756 - precision_31: 0.8860 - recall_31: 0.8632
Epoch 6/50
257/257 [==============================] - 28s 107ms/step - loss: 0.2502 - acc: 0.8980 - precision_31: 0.8986 - recall_31: 0.8981
Epoch 7/50
257/257 [==============================] - 28s

257/257 [==============================] - 28s 110ms/step - loss: 0.1248 - acc: 0.9546 - precision_35: 0.9556 - recall_35: 0.9531
Epoch 11/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0978 - acc: 0.9648 - precision_35: 0.9646 - recall_35: 0.9648
Epoch 12/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0804 - acc: 0.9715 - precision_35: 0.9698 - recall_35: 0.9731
Epoch 13/50
257/257 [==============================] - 28s 110ms/step - loss: 0.0715 - acc: 0.9752 - precision_35: 0.9746 - recall_35: 0.9756
Epoch 14/50
257/257 [==============================] - 28s 110ms/step - loss: 0.0579 - acc: 0.9810 - precision_35: 0.9802 - recall_35: 0.9817
Epoch 15/50
257/257 [==============================] - 28s 111ms/step - loss: 0.0754 - acc: 0.9742 - precision_35: 0.9722 - recall_35: 0.9761
Epoch 16/50
257/257 [==============================] - 28s 110ms/step - loss: 0.0452 - acc: 0.9860 - precision_35: 0.9854 - recall_35: 0.9866
Epoch 17/50
257/25

257/257 [==============================] - 28s 110ms/step - loss: 0.0444 - acc: 0.9882 - precision_36: 0.9889 - recall_36: 0.9877
Epoch 33/50
257/257 [==============================] - 28s 110ms/step - loss: 0.0387 - acc: 0.9887 - precision_36: 0.9896 - recall_36: 0.9879
Epoch 00033: early stopping
29/29 [==============================] - 2s 60ms/step - loss: 0.9594 - acc: 0.7853 - precision_36: 0.7256 - recall_36: 0.8629
------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/50
257/257 [==============================] - 28s 109ms/step - loss: 0.5597 - acc: 0.7073 - precision_37: 0.7253 - recall_37: 0.6651
Epoch 2/50
257/257 [==============================] - 28s 109ms/step - loss: 0.4574 - acc: 0.7930 - precision_37: 0.8090 - recall_37: 0.7659
Epoch 3/50
257/257 [==============================] - 28s 109ms/step - loss: 0.3987 - acc: 0.8220 - precision_37: 0.8333 - recall_37: 0.8040
Epoch 4/50
257/257 [==============================] - 

257/257 [==============================] - 28s 109ms/step - loss: 0.0349 - acc: 0.9890 - precision_40: 0.9894 - recall_40: 0.9885
Epoch 22/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0407 - acc: 0.9853 - precision_40: 0.9850 - recall_40: 0.9853
Epoch 23/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0502 - acc: 0.9833 - precision_40: 0.9836 - recall_40: 0.9828
Epoch 24/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0532 - acc: 0.9830 - precision_40: 0.9821 - recall_40: 0.9836
Epoch 25/50
257/257 [==============================] - 28s 110ms/step - loss: 0.0172 - acc: 0.9944 - precision_40: 0.9939 - recall_40: 0.9949
Epoch 26/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0280 - acc: 0.9906 - precision_40: 0.9892 - recall_40: 0.9919
Epoch 27/50
257/257 [==============================] - 28s 109ms/step - loss: 0.0212 - acc: 0.9944 - precision_40: 0.9946 - recall_40: 0.9941
Epoch 28/50
257/25

In [46]:
def Average(lst):
    return sum(lst) / len(lst)

In [47]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.782377
test precision: 0.757419
test recall: 0.833122
test f1_score: 0.792696


In [ ]:
#glove

In [19]:
df

,tweets,duygu
0,yine mi cuma?????? (@ turkcell küçükyalı plaza...,1
1,lan olm ya ders seçerken net giderse neticede ...,1
2,turkcell sağolsun 4 gb internet hediye etti ar...,1
3,turkcell bin 799 köyü 4.5g ile tanıştıracak,1
4,"ah turkcell ah, beni bu şapşal uygulamalarınla...",0
...,...,...
9121,o değil artık turkcell de mesaj atmıyor.,0
9122,orospu çocuğu turkcell,0
9123,"internetsiz kaldim sinirsizdan , sinirliya gec...",0
9124,geçtiğimiz yıl serdar ortaç'ın bir şarkısını t...,0


In [20]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [21]:
X_t=df['tweets'].tolist()

In [22]:
# Importing library
from keras.preprocessing.text import Tokenizer
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [23]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(X_t, window=5)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=32, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=50, no_threads=8, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('kemik_glove.model')

Performing 50 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49


In [24]:
wordembeddings=glove.load('kemik_glove.model')

In [25]:
def glove (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 32 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [26]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= glove (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn
    early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=256, kernel_size=5, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=2)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(360, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(300, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(260, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(150, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(120, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(80, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(1, activation='sigmoid'))
    modelCNN.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNN.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 9s 33ms/step - loss: 0.6667 - acc: 0.5885 - precision_20: 0.5909 - recall_20: 0.5684
Epoch 2/50
257/257 [==============================] - 9s 34ms/step - loss: 0.6377 - acc: 0.6324 - precision_20: 0.6729 - recall_20: 0.5117
Epoch 3/50
257/257 [==============================] - 9s 33ms/step - loss: 0.6247 - acc: 0.6429 - precision_20: 0.6873 - recall_20: 0.5210
Epoch 4/50
257/257 [==============================] - 9s 33ms/step - loss: 0.6128 - acc: 0.6548 - precision_20: 0.6854 - recall_20: 0.5691
Epoch 5/50
257/257 [==============================] - 8s 32ms/step - loss: 0.6070 - acc: 0.6565 - precision_20: 0.6815 - recall_20: 0.5845
Epoch 6/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5922 - acc: 0.6732 - precision_20: 0.7092 - recall_20: 0.5842
Epoch 7/50
257/257 [==============================] - 8s 32ms/step - 

257/257 [==============================] - 9s 36ms/step - loss: 0.5707 - acc: 0.6840 - precision_22: 0.7045 - recall_22: 0.6318
Epoch 12/50
257/257 [==============================] - 9s 35ms/step - loss: 0.5654 - acc: 0.6921 - precision_22: 0.7114 - recall_22: 0.6442
Epoch 13/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5626 - acc: 0.6950 - precision_22: 0.7042 - recall_22: 0.6703
Epoch 14/50
257/257 [==============================] - 8s 32ms/step - loss: 0.5512 - acc: 0.7086 - precision_22: 0.7251 - recall_22: 0.6701
Epoch 15/50
257/257 [==============================] - 9s 34ms/step - loss: 0.5498 - acc: 0.7055 - precision_22: 0.7192 - recall_22: 0.6720
Epoch 16/50
257/257 [==============================] - 8s 33ms/step - loss: 0.5419 - acc: 0.7162 - precision_22: 0.7311 - recall_22: 0.6820
Epoch 17/50
257/257 [==============================] - 9s 33ms/step - loss: 0.5343 - acc: 0.7147 - precision_22: 0.7286 - recall_22: 0.6825
Epoch 18/50
257/257 [===========

257/257 [==============================] - 8s 33ms/step - loss: 0.4911 - acc: 0.7554 - precision_24: 0.7753 - recall_24: 0.7209
Epoch 24/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4902 - acc: 0.7526 - precision_24: 0.7799 - recall_24: 0.7056
Epoch 25/50
257/257 [==============================] - 9s 33ms/step - loss: 0.4869 - acc: 0.7502 - precision_24: 0.7669 - recall_24: 0.7207
Epoch 26/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4801 - acc: 0.7568 - precision_24: 0.7746 - recall_24: 0.7263
Epoch 27/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4753 - acc: 0.7589 - precision_24: 0.7779 - recall_24: 0.7265
Epoch 28/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4596 - acc: 0.7663 - precision_24: 0.7856 - recall_24: 0.7343
Epoch 29/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4649 - acc: 0.7594 - precision_24: 0.7739 - recall_24: 0.7348
Epoch 30/50
257/257 [===========

257/257 [==============================] - 9s 33ms/step - loss: 0.4147 - acc: 0.7927 - precision_26: 0.7907 - recall_26: 0.7959
Epoch 37/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4120 - acc: 0.7895 - precision_26: 0.7890 - recall_26: 0.7903
Epoch 38/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4135 - acc: 0.7958 - precision_26: 0.8004 - recall_26: 0.7881
Epoch 39/50
257/257 [==============================] - 9s 34ms/step - loss: 0.4058 - acc: 0.8006 - precision_26: 0.8014 - recall_26: 0.7991
Epoch 40/50
257/257 [==============================] - 9s 35ms/step - loss: 0.4038 - acc: 0.8000 - precision_26: 0.7955 - recall_26: 0.8074
Epoch 41/50
257/257 [==============================] - 9s 35ms/step - loss: 0.4054 - acc: 0.7978 - precision_26: 0.8065 - recall_26: 0.7835
Epoch 42/50
257/257 [==============================] - 9s 34ms/step - loss: 0.3922 - acc: 0.8062 - precision_26: 0.8090 - recall_26: 0.8015
Epoch 43/50
257/257 [===========

257/257 [==============================] - 8s 33ms/step - loss: 0.3738 - acc: 0.8179 - precision_28: 0.8263 - recall_28: 0.8075
Epoch 50/50
29/29 [==============================] - 0s 9ms/step - loss: 0.6736 - acc: 0.6743 - precision_28: 0.6997 - recall_28: 0.5530
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/50
257/257 [==============================] - 9s 35ms/step - loss: 0.6700 - acc: 0.5885 - precision_29: 0.5861 - recall_29: 0.5950
Epoch 2/50
257/257 [==============================] - 9s 35ms/step - loss: 0.6387 - acc: 0.6328 - precision_29: 0.6651 - recall_29: 0.5310
Epoch 3/50
257/257 [==============================] - 9s 36ms/step - loss: 0.6285 - acc: 0.6382 - precision_29: 0.6882 - recall_29: 0.5017
Epoch 4/50
257/257 [==============================] - 9s 35ms/step - loss: 0.6109 - acc: 0.6539 - precision_29: 0.6900 - recall_29: 0.5554
Epoch 5/50
257/257 [==============================] - 9s 34ms/step - loss: 0.6007

In [27]:
def Average(lst):
    return sum(lst) / len(lst)

In [28]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.666558
test precision: 0.687012
test recall: 0.621789
test f1_score: 0.649048


In [29]:
#bert

In [7]:
import math
import bert
from tensorflow.keras import layers
from transformers import AutoModel, AutoTokenizer
import re
import random

In [8]:
df

,tweets,duygu
0,turkcell'den elimi ayağımı çektiğim iyi oldu,0
1,"avrasya tüneli inşaatı sırasında, yerin 106 me...",1
2,salla kazan yeniden aramızda ?????? hazırkart ...,1
3,turkcell mukemmele yakin kalitede musteri tems...,1
4,o deilde bugün turkcell hiç msj atmadı amk kes...,0
...,...,...
9121,oha bu turkcell reklamındaki bıdıkları yerim y...,1
9122,ulan turkcell lazım olduğunda dakika gönderme ...,0
9123,en beğenilen şirket ödülü yine turkcell'in,1
9124,can yücel: bağlanmayacaksın. turkcell superonl...,0


In [9]:
tweets = list(df['tweets'])


In [10]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True)

In [11]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [12]:
#rnn
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=256,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.3,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.rnn_layer1 = layers.SimpleRNN(128,
                                        return_sequences= True,
                                        activation="relu")
        self.rnn_layer2 = layers.SimpleRNN(256,
                                        return_sequences= True,
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.rnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.rnn_layer2(l) 
        l_2 = self.pool(l_2)
        
        concatenated = tf.concat([l_1,l_2], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [13]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 50
early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=True)

In [14]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [15]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif


In [24]:
train_data,test_data,processed_dataset_test

(<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>,
 <PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>,
 <FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>)

In [18]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=2, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
BATCH_SIZE = 16

for train_index, test_index in kfold.split(df):
    # splitting Dataframe (dataset not included)
    
    train_df = df.iloc[train_index].tweets.tolist()
    test_df = df.iloc[test_index].tweets.tolist()
    train_y = np.array(df.iloc[train_index].duygu.tolist())
    test_y = np.array(df.iloc[test_index].duygu.tolist())

    tokenized_reviews_train = [tokenize_reviews(tweet) for tweet in train_df]
    tokenized_reviews_test = [tokenize_reviews(tweet) for tweet in test_df] 
    
    reviews_with_len_train = [[review, train_y[i], len(review)]
                     for i, review in enumerate(tokenized_reviews_train)]
    
    reviews_with_len_test = [[review, test_y[i], len(review)]
                     for i, review in enumerate(tokenized_reviews_test)]

    random.shuffle(reviews_with_len_train)
    random.shuffle(reviews_with_len_test)
    
    reviews_with_len_train.sort(key=lambda x: x[2])
    reviews_with_len_test.sort(key=lambda x: x[2])
    
    sorted_reviews_train = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len_train]
    sorted_reviews_test = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len_test]
    
    processed_dataset_test = tf.data.Dataset.from_generator(lambda: sorted_reviews_test, output_types=(tf.int32, tf.int32))
    processed_dataset_train = tf.data.Dataset.from_generator(lambda: sorted_reviews_train, output_types=(tf.int32, tf.int32))

    test_data = processed_dataset_test.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
    train_data = processed_dataset_train.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    text_model.fit(train_data, epochs=1,callbacks=early_stopping)
    
    loss, accuracy, precision, recall = text_model.evaluate(test_data)

    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    # Increase fold number
    fold_no = fold_no + 1

["salla kazan yeniden aramızda ?????? hazırkart veya faturalı olmak farketmiyor numarasını turkcell'e taşıyan...", "4 senelik vodafone kullanicisi olarak turkcell musteri hizmetlerini aradim, vodafone da boyle samimiyet gormedim. turkcell'e gececegim :)# turkcell de yeni 4.5g li internet+ses+sms paketlerde 2 kat internet var. 3 gb internetli paket aldim + 3 gb daha geldi supermis :)", "turkcell'e ve gırtlağıma yatırdığım paraları biriktirsem ali ağaoğlu'nu masseratimle çoktan sollamıştım.", 'bugün avea ile olan sözleşmemi 1 ay daha uzattım. yine kapamadın beni turkcell.', "@turkcell 4.5g hiziyla internete kolayca baglanmak, kamerasiyla guzel fotograflar cekmek, turkcell'le yine gurur duymak icin #t70benimolsun", 'neye ihtiyacınız yoksa onu hediye eden kuruma turkcell denir.', 'turkcell sana deli gb fatura ödüyorum ama dandiklikten ölüceksin bigün! pişmanlığımsın.', "zencilere ankara havası  oynatan turkcell'e saygılar :d çok iyi reklam olmuş. @turkcell", 'turkcell t(60)aldim almaz olay

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



286/286 [==============================] - 2s 7ms/step - loss: 0.0601 - acc: 0.9801 - precision: 0.9955 - recall: 0.9650
["turkcell'den elimi ayağımı çektiğim iyi oldu", 'avrasya tüneli inşaatı sırasında, yerin 106 metre altında tüm mobil haberleşmeyi sağlayan sadece turkcell oldu.…', 'turkcell mukemmele yakin kalitede musteri temsilcileri cok iyi,cozum uretiyor heryerde cekiyor fakat pahali ee okdr hizmete deger', 'o deilde bugün turkcell hiç msj atmadı amk kesin başka biri var :(', 'turkcell mesajlarımı iletmiyo amk. sizin ben amınıza koyayım orospu çocukları', 'vodafone geciyorum diye turkcell trip atiyo , son gunlerimde internetimi yavaslatti :d', 'turkcell timsah arena, yakıştı valla', 'turkcell derdin her neyse benimle adam gibi iletişime geç, öbür türlü mesajın bana ulaşmadan siliniyor.', 'abi bnm telefon gece 00:00 dan sonra başlıyo çalışmaya gündüz turkcell(türksel)den başka kimse atmaz gecede maşallah susmaz :d', 'turkcell bin 799 köyü 4.5g ile tanıştıracak', '@turkcell turkc

["salla kazan yeniden aramızda ?????? hazırkart veya faturalı olmak farketmiyor numarasını turkcell'e taşıyan...", "4 senelik vodafone kullanicisi olarak turkcell musteri hizmetlerini aradim, vodafone da boyle samimiyet gormedim. turkcell'e gececegim :)# turkcell de yeni 4.5g li internet+ses+sms paketlerde 2 kat internet var. 3 gb internetli paket aldim + 3 gb daha geldi supermis :)", "turkcell'e ve gırtlağıma yatırdığım paraları biriktirsem ali ağaoğlu'nu masseratimle çoktan sollamıştım.", 'bugün avea ile olan sözleşmemi 1 ay daha uzattım. yine kapamadın beni turkcell.', "@turkcell 4.5g hiziyla internete kolayca baglanmak, kamerasiyla guzel fotograflar cekmek, turkcell'le yine gurur duymak icin #t70benimolsun", 'neye ihtiyacınız yoksa onu hediye eden kuruma turkcell denir.', 'turkcell sana deli gb fatura ödüyorum ama dandiklikten ölüceksin bigün! pişmanlığımsın.', "zencilere ankara havası  oynatan turkcell'e saygılar :d çok iyi reklam olmuş. @turkcell", 'turkcell t(60)aldim almaz olay

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



286/286 [==============================] - 2s 7ms/step - loss: 0.0210 - acc: 0.9939 - precision: 0.9912 - recall: 0.9964


In [17]:
def Average(lst):
    return sum(lst) / len(lst)

In [18]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.973492
test precision: 0.977200
test recall: 0.968627
test f1_score: 0.972796
